In [1]:
# plotting
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# numeric
import quandl
import numpy as np
from numpy import random
import pandas as pd

# util
from collections import Counter
import pdb
import time
import tempfile
import itertools
from tqdm import tqdm_notebook as tqdm

# logging
import logging
logger = log = logging.getLogger(__name__)
# log.setLevel(logging.INFO)
logging.basicConfig()
log.info('%s logger started.', __name__)

In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

In [3]:
import os
os.sys.path.append(os.path.abspath('.'))
# %reload_ext autoreload
# %autoreload 2

In [4]:
from src.environments.portfolio import PortfolioEnv

In [5]:
class EnvWrapper(PortfolioEnv):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def step(self, action):
        # also it puts it in a list
        if isinstance(action, list):
            action = action[0]
        
        # we have to normalise for some reason softmax wont work
        if isinstance(action, dict):
            action = np.abs(list(action.values()))
            action /= action.sum()        
        
        return super().step(action) 

In [6]:
df_train = pd.read_hdf('./data/poliniex_30m.hf',key='train')
env = EnvWrapper(
    df=df_train,
    steps=128, 
    scale=True, 
    augument=0.0005    
)
env.seed = 0   

df_test = pd.read_hdf('./data/poliniex_30m.hf',key='test')
env_test = EnvWrapper(
    df=df_test,
    steps=128, 
    scale=True, 
    augument=0.00)
env_test.seed = 0  

In [7]:
from tensorforce.environments.openai_gym import OpenAIGym
environment = OpenAIGym('CartPole-v0')
environment.gym = env

INFO:gym.envs.registration:Making new env: CartPole-v0
[2017-07-15 16:58:41,179] Making new env: CartPole-v0


In [8]:
from tensorforce import Configuration
from tensorforce.agents import NAFAgent
from tensorforce.core.networks import layered_network_builder

In [9]:
# Define a network builder from an ordered list of layers
# https://github.com/reinforceio/tensorforce/blob/0d07fadec03f76537a2431e17c51cd759d53b5e9/tensorforce/core/networks/layers.py
layers = [
    dict(type='flatten'),
    dict(type='dense', size=32, l2_regularization=1e-8, activation='relu'),
    dict(type='dense', size=32, l2_regularization=1e-8, activation='relu'),    
]
# act will it add's it's own head so we can't add a softmax at the end
network = layered_network_builder(layers_config=layers)


In [10]:
tuple(env.observation_space.shape), env.action_space.shape[0]

((6, 8), 6)

In [12]:

config = Configuration(
    # Agent
    network=network,  
    states=dict(shape=tuple(env.observation_space.shape), type='float'),
    actions={'action' + str(n): dict(continuous=True) for n in range(env.action_space.shape[0])},
    exploration=dict(
        type='EpsilonDecay',
        kwargs=dict(epsilon=1, epsilon_final=0.01, epsilon_timesteps=1e4)
    ),
    preprocessing=None,
    
    # The `MemoryAgent` class additionally requires the following parameters:
    batch_size=32, # integer of the batch size.
    memory_capacity=50, # integer of maximum experiences to store.
    memory = 'prioritized_replay', # string indicating memory type ('replay' or 'prioritized_replay').
#     memory_args= # list of arguments to pass to replay memory constructor.
#     memory_kwargs= # list of keyword arguments to pass to replay memory constructor.
    update_frequency=10, # integer indicating the number of steps between model updates.
#     first_update= # integer indicating the number of steps to pass before the first update.
    repeat_update=10, # integer indicating how often to repeat the model update.
    
    # Each model requires the following configuration parameters:
    discount=0.99, # float of discount factor (gamma).
    learning_rate=0.001, # float of learning rate (alpha).
    optimizer='adam', # string of optimizer to use (e.g. 'adam').
#     optimizer_args= # list of arguments for optimizer.
#     optimizer_kwargs= # dict of keyword arguments for optimizer.
#     device= # string of tensorflow device name.
#     tf_saver= # boolean whether to save model parameters.
#     tf_summary= # boolean indicating whether to use tensorflow summary file writer.
#     log_level="debug" # string containing loglevel (e.g. 'info').
#     distributed= # boolean indicating whether to use distributed tensorflow.
#     global_model= # global model.
#     session= # session to use.

    # The NAF agent expects the following additional configuration parameters:
    target_update_frequency = 1, # int of states between updates of the target network.
    update_target_weight = 0.001, # float of update target weight (tau parameter).
    clip_gradients = 10, # float of maximum values for gradients before clipping.
)

# Create a Trust Region Policy Optimization agent
agent = NAFAgent(config=config)

# for some reason these are not set?
# agent.next_internal = agent.current_internal = []

ValueError: Dimensions must be equal, but are 6 and 5 for 'training_outputs/add' (op: 'Add') with input shapes: [?,6,6], [?,5,6].

In [ ]:
# why does softmax not work, how to view this?
# agent.model

In [ ]:

# Poll new state from client
state = environment.reset()

for i in range(10):
    # Get prediction from agent, execute
    action = agent.act(state=state)

    state, reward, done = environment.execute(action)

    # Add experience, agent automatically updates model according to batch size
    agent.observe(reward=reward, terminal=False)
    
    a=np.array(list(action.values()))
    print(a, a.sum(),a.min(),a.max())

In [ ]:
from tensorforce.execution import Runner
runner = Runner(agent=agent, environment=environment)

In [ ]:
# Callback function printing episode statistics
def episode_finished(r):
    log_intv = 200
    if r.episode % log_intv == 0:
        df = pd.DataFrame(env.sim.infos)
        print(
            "Finished episode {ep} after {ts} timesteps (reward: {reward: 2.4f}) portfolio_value: {portfolio_value: 2.4f}".
            format(
                ep=r.episode,
                ts=r.timestep,
                reward=r.episode_rewards[-log_intv],
                portfolio_value=df.portfolio_value[-1:].mean()
            )
        )
    return True

In [ ]:
runner.run(
    episodes=90000, max_timesteps=200, episode_finished=episode_finished)